In [45]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [46]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.4.1
tiktoken version: 0.7.0


# 1.1 Tokenizing text

In [47]:
with open("/home/bakayu/projects/work/sadaak_htm/sadaakAI/.workshop/LLM-workshop-2024/02_data/the-verdict.txt", "r") as f:
    raw_text = f.read()

print("total number of characters:", len(raw_text))
print(raw_text[:99])

total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [48]:
import re

preprocessed = re.split(r'([,.:;?_!"{}\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item]
print(preprocessed[:99])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that', ',', ' ', 'in', ' ', 'the', ' ', 'height', ' ', 'of', ' ', 'his', ' ', 'glory', ',', ' ', 'he', ' ', 'had', ' ', 'dropped', ' ', 'his', ' ', 'painting', ',', ' ', 'married', ' ', 'a', ' ', 'rich', ' ', 'widow', ',', ' ', 'and', ' ', 'established', ' ', 'himself', ' ', 'in', ' ', 'a', ' ', 'villa', ' ', 'on']


In [49]:
print("no of tokens:", len(preprocessed))

no of tokens: 8400


# 1.2 Coverting tokens into token IDs

In [50]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1133


In [51]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [52]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('\n', 0)
(' ', 1)
('!', 2)
('"', 3)
("'", 4)
('(I', 5)
('(Though', 6)
(')', 7)
(',', 8)
('--', 9)
('.', 10)
(':', 11)
(';', 12)
('?', 13)
('A', 14)
('Ah', 15)
('Among', 16)
('And', 17)
('Are', 18)
('Arrt', 19)
('As', 20)
('At', 21)
('Be', 22)
('Begin', 23)
('Burlington', 24)
('But', 25)
('By', 26)
('Carlo', 27)
('Chicago', 28)
('Claude', 29)
('Come', 30)
('Croft', 31)
('Croft)', 32)
('Destroyed', 33)
('Devonshire', 34)
('Don', 35)
('Dubarry', 36)
('Emperors', 37)
('Florence', 38)
('For', 39)
('Gallery', 40)
('Gideon', 41)
('Gisburn', 42)
('Gisburns', 43)
('Grafton', 44)
('Greek', 45)
('Grindle', 46)
('Grindles', 47)
('HAD', 48)
('Had', 49)
('Hang', 50)


In [53]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [54]:
tokenizer = SimpleTokenizerV1(vocab)

text = """
"It's the last he painted, you know,"
Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[3, 60, 4, 853, 991, 605, 536, 749, 8, 1129, 599, 8, 3, 71, 10, 42, 854, 1111, 757, 796, 10]


In [55]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [56]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

# 1.3 BytePair encoding

In [57]:
import importlib
import importlib.metadata
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [58]:
tokenizer = tiktoken.get_encoding("gpt2")

In [59]:
text = (
    "I Hurt, Therefore I Am. <|endoftext|> To be, is to be percieved"
    "We are their flaws."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[40, 42566, 11, 8447, 314, 1703, 13, 220, 50256, 1675, 307, 11, 318, 284, 307, 583, 66, 39591, 1135, 389, 511, 17978, 13]


In [60]:
strings = tokenizer.decode(integers)

print(strings)

I Hurt, Therefore I Am. <|endoftext|> To be, is to be percievedWe are their flaws.


In [61]:
shadow_slave_string = tokenizer.encode("TheTreach rous shadow", allowed_special={"<|endoftext|>"})
# shadow_slave_string = tokenizer.decode(shadow_slave_string)
print(shadow_slave_string)

[464, 51, 16250, 374, 516, 9082]


# 1.4 Data sampling with a sliding window

In [62]:
from supplementary import create_dataloader_v1


dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
